In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from bs4 import BeautifulSoup
import requests 

# Compiling mutual fund data from all pages of Groww Investment platform

In [6]:
final=pd.DataFrame(columns=['Mutual Fund Name','fund_rating','return_1yr',
      'return_3yr','return_5yr',
      'risk_type','category'])

final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Mutual Fund Name  0 non-null      object
 1   fund_rating       0 non-null      object
 2   return_1yr        0 non-null      object
 3   return_3yr        0 non-null      object
 4   return_5yr        0 non-null      object
 5   risk_type         0 non-null      object
 6   category          0 non-null      object
dtypes: object(7)
memory usage: 0.0+ bytes


In [9]:
for j in range(73):
    url='https://groww.in/mutual-funds/filter?q=&fundSize=&pageNo={}&sortBy=0'.format(j)
    page=requests.get(url)
    soup=BeautifulSoup(page.content,'html.parser')
 
  #  fund containers
    fund=soup.find_all('tr',class_="f22Card")

  #scraping all field from each section
    name=[]
    rating=[]
    return_1yr=[]
    return_3yr=[]
    return_5yr=[]
    risk_type=[]
    category=[]
    

    for i in fund:
        name.append(i.find('div',class_="fs14 clrText fw500 f22LH34 f22Mb4 truncate").text.strip())
        return_1yr.append(i.find_all('div',class_="fs14 clrText fw500 center-align f22Mb4")[0].text.strip())
        return_3yr.append(i.find_all('div',class_="fs14 clrText fw500 center-align f22Mb4")[1].text.strip())
        return_5yr.append(i.find_all('div',class_="fs14 clrText fw500 center-align f22Mb4")[2].text.strip())
        risk_type.append(i.find('div',class_="fs12 fw500 clrSubText f22Ls2").text.strip())
        category.append(i.find('div',class_="fs12 fw500clrSubText f22Ls2").text.strip())
        try: 
              rating.append(i.find('div',class_="fs12 fw500 clrSubText f22Ls2 valign-wrapper").text.strip())
        except:
              rating.append(np.nan)

    d={'Mutual Fund Name':name,'fund_rating':rating,'return_1yr':return_1yr,
      'return_3yr':return_3yr,'return_5yr':return_5yr,
      'risk_type':risk_type,'category':category} 
    
    df=pd.DataFrame(d)

    #combining dataframes from each page
    final = pd.concat([final,df],ignore_index=True)  

In [12]:
final.to_parquet('mutual_funds.gzip', compression='gzip')

# Get Historical Data

In [23]:
daterange = pd.date_range(start="2020-04-30", end="2023-04-30", freq='B').strftime('%d-%b-%Y').tolist() 


In [ ]:
from IPython.display import clear_output
from io import StringIO
final = pd.DataFrame(columns=['Scheme Code','Scheme Name','ISIN Div Payout/ISIN Growth','ISIN Div Reinvestment','Net Asset Value','Repurchase Price','Sale Price','Date'])

for date in daterange:
    clear_output(wait=True)
    hurl = f'https://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?tp=1&frmdt={date}'
    data = requests.get(hurl)
    file1 = []
    for line in data.text.split('\n'):
        if line.find(';')!=-1: file1.append(line)
        else: continue 
    ssvfile = '\n'.join(file1)
    df = pd.read_csv(StringIO(ssvfile), sep=";")
    final = pd.concat([final, df], ignore_index=True)
    print(date)
    